In [ ]:
from __future__ import print_function
import os
import subprocess
import time

from astropy.io import ascii
from astropy.io import fits
from astropy.nddata import Cutout2D
from astropy.stats import sigma_clipped_stats
from astropy.table import Table
from astropy import units as u
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization import astropy_mpl_style

import image_registration

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import numpy as np

import pandas as pd
import pandasql as ps

from PIL import Image
import PIL.ImageOps    

import scipy.misc

plt.style.use(astropy_mpl_style)



In [2]:
def makeFits(inName,normalVal='none',imageExtension='.jpg'):
    image = Image.open(inName)
    xsize, ysize = image.size
    print("Image size: {} x {}".format(xsize, ysize))
    # plt.imshow(image)

    r, g, b = image.split()
    r_data = np.array(r.getdata()) # data is now an array of length ysize*xsize
    g_data = np.array(g.getdata())
    b_data = np.array(b.getdata())
    print(r_data.shape)

    r_data = r_data.reshape(ysize, xsize)[::-1]
    g_data = g_data.reshape(ysize, xsize)[::-1]
    b_data = b_data.reshape(ysize, xsize)[::-1]

    inName = inName.replace(imageExtension,'')
    print(inName)
    try:
        os.remove(inName+"red.fits")
        os.remove(inName+"green.fits")
        os.remove(inName+"blue.fits")
        os.remove(inName+"average.fits")
    except:
        print("Can't remove as the file doesn't exist")

    r_data_cropped = cropImage(r_data) # remove timestamp
    if normalVal != 'none':
        r_data_cropped = normaliseImage(r_data_cropped,normalVal) # noramlisation
    red = fits.PrimaryHDU(r_data_cropped)
    hdur = fits.HDUList([red])
    hdur.data = np.int16(hdur.data) # 16 bit transformation
    hdur.writeto(inName+'red.fits')
    
    g_data_cropped = cropImage(g_data) # remove timestamp
    if normalVal != 'none':
        g_data_cropped = normaliseImage(g_data_cropped,normalVal) # noramlisation
    green = fits.PrimaryHDU(g_data_cropped)
    hdug = fits.HDUList([green])
    hdug.data = np.int16(hdug.data) # 16 bit transformation
    hdug.writeto(inName+'green.fits')

    b_data_cropped = cropImage(b_data)
    if normalVal != 'none':
        b_data_cropped = normaliseImage(b_data_cropped,normalVal) # noramlisation
    blue = fits.PrimaryHDU(b_data_cropped)
    hdub = fits.HDUList([blue])
    
    hdub.data = np.int16(hdub.data) # 16 bit transformation
    hdub.writeto(inName+'blue.fits')
    
    average_data = (r_data_cropped + g_data_cropped + b_data_cropped)/3.0 # A complete average is not necessarily the best for mole detection
    average = fits.PrimaryHDU(average_data)
    hduA = fits.HDUList([average])
    hduA.data = np.int16(hduA.data)
    hduA.writeto(imName+'average.fits')